# Open

In [ ]:
!nvidia-smi

Fri May 20 08:55:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd drive/MyDrive/komi;ls

data  df_train.csv  df_val.csv	README.txt  train.csv  Медиатека_коми_рус.xlsx


# t5

## Models to try
### google/mt5-small
### cointegrated/rut5-small
### sberbank-ai/ruT5-base

In [3]:
!pip install transformers
!pip install datasets==1.15.1
!pip install sentencepiece

     |████████████████████████████████| 4.2 MB 9.4 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 8.3 MB/s 
     |████████████████████████████████| 212 kB 67.7 MB/s 
     |████████████████████████████████| 140 kB 77.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.8 MB/s 
     |████████████████████████████████| 144 kB 71.1 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 77.6 MB/s 
     |████████████████████████████████| 1.2 MB 8.0 MB/s 


In [ ]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer

# model = 'google/mt5-base'
model = 'sberbank-ai/ruT5-large' # ('google/mt5-small')('cointegrated/rut5-small')('sberbank-ai/ruT5-base')
tokenizer = T5Tokenizer.from_pretrained(model)
# 
# train_dataset = train_dataset.map(lambda e: tokenizer(e['komi'], padding='max_length', truncation=True), batched=True)
# train_dataset = train_dataset.rename_columns({'attention_mask': 'attention_mask_train', 'input_ids': 'input_ids_train', 'komi':'komi', 'rus':'rus'})
# train_dataset = train_dataset.map(lambda e: tokenizer(e['rus'], padding='max_length', truncation=True), batched=True)
# train_dataset = train_dataset.rename_columns({'attention_mask': 'attention_mask_label', 'input_ids': 'input_ids_label', 'komi':'komi', 'rus':'rus'})
# train_dataset.set_format(type='torch', columns=['attention_mask_label', 'attention_mask_train', 'input_ids_train', 'input_ids_label'])
# 
# val_dataset = val_dataset.map(lambda e: tokenizer(e['komi'], padding='max_length', truncation=True), batched=True)
# val_dataset = val_dataset.rename_columns({'attention_mask': 'attention_mask_train', 'input_ids': 'input_ids_train', 'komi':'komi', 'rus':'rus'})
# val_dataset = val_dataset.map(lambda e: tokenizer(e['rus'], padding='max_length', truncation=True), batched=True)
# val_dataset = val_dataset.rename_columns({'attention_mask': 'attention_mask_label', 'input_ids': 'input_ids_label', 'komi':'komi', 'rus':'rus'})
# val_dataset.set_format(type='torch', columns=['attention_mask_label', 'attention_mask_train', 'input_ids_train', 'input_ids_label'])
# 
# -100 - специальное значение, позволяющее не учитывать токены
#       y.input_ids[y.input_ids == 0] = -100

Downloading:   0%|          | 0.00/980k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

In [ ]:

from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained(model).cuda()

# checkpoint = torch.load('drive/MyDrive/komi/data/sber_t5_large_adm_media_3epocs.pth')
# model.load_state_dict(checkpoint['model_state_dict'])

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

In [ ]:
import torch 
from torch.optim import Adam

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [16]:
import pandas as pd

# # train_example_2
# df_train = pd.read_csv('drive/MyDrive/komi/data/train_example_2/df_train.csv')
# df_val = pd.read_csv('drive/MyDrive/komi/data/train_example_2/df_val.csv')
# 
# pairs = list(zip(df_train['komi'].tolist(), df_train['rus'].tolist()))
# pairs[0], len(df_train)

# train_example_3
df = pd.read_csv('drive/MyDrive/komi/data/train_example_3/final_two_variants_dataset.csv')
df = df.sample(frac=1, random_state=1)
df['len'] = df.apply(lambda x: len(x['target_text'].split()), axis=1)
df = df[df['len'] < 60][df.columns[:-1]]
df_train = df.iloc[:int(len(df) * 0.8)]
pairs = list(zip(df_train['input_text'].tolist(), df_train['target_text'].tolist()))
pairs[0], len(df_train)

(('Федосовой Ольге Робертовне – директору Печорского промышленно-экономического техникума.',
  'Ольга Робертовна Федосовалы – Печораса промышленно-экономическӧй техникумса директорлы.'),
 64372)

### Train

In [ ]:
from tqdm.auto import trange
import random
import numpy as np

batch_size = 2
report_steps = 200
epochs = 3
# epochs = checkpoint['epoch']

model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    # random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        # -100 - специальное значение, позволяющее не учитывать токены
        y.input_ids[y.input_ids == 0] = -100

        # loss = checkpoint['loss']
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 0


  0%|          | 0/32186 [00:00<?, ?it/s]

step 0 loss 7.203848838806152
step 200 loss 5.8963279688358305
step 400 loss 4.7509380042552944
step 600 loss 4.534606263637543
step 800 loss 4.347838731408119
step 1000 loss 4.254292127490044
step 1200 loss 4.060615223646164
step 1400 loss 3.9622719466686247
step 1600 loss 3.9335459977388383
step 1800 loss 3.8113812655210495
step 2000 loss 3.8598673510551453
step 2200 loss 3.647377904653549
step 2400 loss 3.630960738062859
step 2600 loss 3.572526582479477
step 2800 loss 3.5078496915102004
step 3000 loss 3.5557223284244537
step 3200 loss 3.438501150012016
step 3400 loss 3.405920510292053
step 3600 loss 3.385431077480316
step 3800 loss 3.2297156971693037
step 4000 loss 3.312857437133789
step 4200 loss 3.265197672247887
step 4400 loss 3.17307895720005
step 4600 loss 3.15394039735198
step 4800 loss 3.1548472058773043
step 5000 loss 3.20654703438282
step 5200 loss 3.2006086015701296
step 5400 loss 3.0829707407951354
step 5600 loss 3.1012952549755575
step 5800 loss 2.9912944331765177
step 6

  0%|          | 0/32186 [00:00<?, ?it/s]

step 0 loss 2.0535247626900675
step 200 loss 2.0797463248670103
step 400 loss 1.890882162451744
step 600 loss 1.9223087573796511
step 800 loss 2.0376080933585765
step 1000 loss 2.001545876339078
step 1200 loss 1.9095158122293652
step 1400 loss 1.8790137334167958
step 1600 loss 2.1169552559405567
step 1800 loss 1.8965063777565956
step 2000 loss 2.0321537771821023
step 2200 loss 1.913104035705328
step 2400 loss 1.9360123066604138
step 2600 loss 1.9859614448994398
step 2800 loss 1.9504835142195225
step 3000 loss 2.011861074194312
step 3200 loss 1.954792839884758
step 3400 loss 1.8814445723593236
step 3600 loss 1.92902357628569
step 3800 loss 1.79715731613338
step 4000 loss 1.9001190569065511
step 4200 loss 1.9044659309089185
step 4400 loss 1.8910849940776826
step 4600 loss 1.883146038148552
step 4800 loss 1.8491859897086398
step 5000 loss 1.9443400061689318
step 5200 loss 1.9729149714112282
step 5400 loss 1.8576486627012492
step 5600 loss 1.874662187732756
step 5800 loss 1.841940181627869

  0%|          | 0/32186 [00:00<?, ?it/s]

step 0 loss 1.5741426398977638
step 200 loss 1.5951579797454178
step 400 loss 1.4494823801331223
step 600 loss 1.4987321978807449
step 800 loss 1.5573346148431302
step 1000 loss 1.5667347994446754
step 1200 loss 1.454986178437248
step 1400 loss 1.4002231177687645
step 1600 loss 1.6218125647306443
step 1800 loss 1.4516910610161722
step 2000 loss 1.574324520006776
step 2200 loss 1.4830053568817676
step 2400 loss 1.5011927528027444
step 2600 loss 1.5123755912669004
step 2800 loss 1.5253662542253732
step 3000 loss 1.5784824305400251
step 3200 loss 1.5087469374760984
step 3400 loss 1.4755014511197806
step 3600 loss 1.501233175941743
step 3800 loss 1.3820047084987164
step 4000 loss 1.4650219907425344
step 4200 loss 1.4835855983942747
step 4400 loss 1.4614242116734386
step 4600 loss 1.4675451936945318
step 4800 loss 1.4420434613409452
step 5000 loss 1.5181483852327802
step 5200 loss 1.5508352668583394
step 5400 loss 1.439262208454311
step 5600 loss 1.4927033330872654
step 5800 loss 1.43045180

# Save

# Eval

In [ ]:
model.eval()
torch.save(model.state_dict(), 'drive/MyDrive/komi/data/sber_t5_large_adm_media_3epocs.pth')

In [ ]:
import pandas as pd
from datasets import load_metric

metric = load_metric("bleu")
results = [
           # 'drive/MyDrive/komi/results_sber_t5_large_4ep_08_02_clean_dataset.csv',
           # 'drive/MyDrive/komi/results_mt5_base_4ep_08_02_clean_dataset.csv',
           # 'drive/MyDrive/komi/results_rut5_large_3ep_08_02_cleanDS_admDS.csv'
           # 'drive/MyDrive/komi/results_rut_extra.csv',
           'drive/MyDrive/komi/results_rut.csv'
           ]



for path in results:
  pred = pd.read_csv(path)
  pred['actual'] = df.iloc[int(len(df) * 0.8):]['target_text'].tolist()
  preds = pred['results'].tolist()
  score = metric.compute(predictions=[i.split() if type(i) != float else ''.split() for i in pred], references=[[i.split()] for i in pd.read_csv(path)['actual'].tolist()])
  print(path.split('/')[-1], score['bleu'])
